# Measures

## Default measures

Let's quickly create a cube from a CSV file.
You can see the value of your measures at the top level of the cube by calling `cube.query()`:

In [ ]:
import atoti as tt

session = tt.create_session()
store = session.read_csv("data/example.csv", keys=["ID"], store_name="First")
cube = session.create_cube(store, "FirstCube")
lvl = cube.levels
m = cube.measures
hier = cube.hierarchies
cube.query()

`SUM` and `MEAN` aggregations are automatically created for numeric columns.

Calling `cube.query()` will display the value of the measure at the top level.
It's also possible to specify levels to split the cube:

In [ ]:
cube.query(m["Quantity.SUM"], levels=[lvl["Continent"], lvl["Country"]])

You can also filter on levels:

In [ ]:
cube.query(
    m["Quantity.SUM"],
    levels=[lvl["Continent"], lvl["Country"]],
    condition=(lvl["Country"] == "France"),
)

## Aggregation functions

The available aggregation functions are:

In [ ]:
from inspect import getmembers, isfunction

[member[0] for member in getmembers(tt.agg) if isfunction(member[1])]

In [ ]:
m["Quantity.MIN"] = tt.agg.min(store["Quantity"])
m["Quantity.MAX"] = tt.agg.max(store["Quantity"])
m["Quantity.LONG"] = tt.agg.long(store["Quantity"])
m["Quantity.SHORT"] = tt.agg.short(store["Quantity"])
cube.query(levels=[lvl["Country"]])

In [ ]:
m["Quantity running sum"] = tt.agg.sum(
    m["Quantity.SUM"], scope=tt.scope.cumulative(lvl["ID"])
)
cube.query(m["Quantity running sum"], levels=[lvl["ID"]])

In [ ]:
m["Distinct City"] = tt.agg.count_distinct(store["City"])

In [ ]:
m["Quantile 95"] = tt.agg.quantile(store["Quantity"], 0.95)
m["Quantile 20"] = tt.agg.quantile(store["Quantity"], 0.20)
cube.query(m["Quantile 95"], m["Quantile 20"], levels=[lvl["Continent"]])

Statistics aggregations

In [ ]:
m["Quantity variance"] = tt.agg.var(store["Quantity"], mode="sample")
m["Quantity standard deviation"] = tt.agg.std(store["Quantity"], mode="sample")
cube.query(m["Quantity variance"], m["Quantity standard deviation"])

### MaxMember / MinMember

`max_member` and `min_member` search for the specified extremum of the given measure and return the member of the specified level for which the extremum is reached.

In [ ]:
m["MinMember of Price.SUM"] = tt.agg.min_member(m["Price.SUM"], lvl["City"])
m["MaxMember of Price.SUM"] = tt.agg.max_member(m["Price.SUM"], lvl["City"])

cube.query(m["MinMember of Price.SUM"], m["MaxMember of Price.SUM"])

No matter which level you execute the query on, the argmin is always performed at the level specified when defining the measure

In [ ]:
cube.query(
    m["MinMember of Price.SUM"],
    m["MaxMember of Price.SUM"],
    levels=[lvl["Continent"], lvl["Country"]],
)

## Delete a measure

Any measure can be removed from the cube using `del`

In [ ]:
"Quantity running sum" in m

In [ ]:
del m["Quantity running sum"]
"Quantity running sum" in m

## Different aggregations per level

It's possible to aggregate a column differently depending on whether we are above or below a certain level.

For example, you can sum a quantity for each country and then take the average over all the countries:

In [ ]:
m["Average per country"] = tt.agg.mean(
    m["Quantity.SUM"], scope=tt.scope.origin(lvl["Country"])
)
cube.query(m["Average per country"], levels=[lvl["Country"]])

In [ ]:
cube.query(m["Average per country"])

## Calculated measures

Measures can be combined together to build more complex measures:

In [ ]:
m["Turnover"] = tt.agg.sum(m["Quantity.SUM"] * store["Price"])
cube.query(levels=[lvl["Country"]])

## Multidimensional Functions

### Parent Value

In [ ]:
# Create multi-level hierarchy :
lvl = cube.levels
cube.hierarchies["Geography"] = {
    "continent": lvl["Continent"],
    "country": lvl["Country"],
    "city": lvl["City"],
}

In [ ]:
help(tt.parent_value)

In [ ]:
m["Parent quantity"] = tt.parent_value(m["Quantity.SUM"], on=hier["Geography"])

We can check that the value on countries is equal to the `Quantity.SUM` on their continent :

In [ ]:
cube.query(m["Parent quantity"], levels=[lvl["continent"], lvl["country"]])

In [ ]:
cube.query(m["Quantity.SUM"], levels=[lvl["continent"]])

### Aggregate siblings

Siblings are all the children of the same member. For instance France and UK are children of Europe so France and UK are siblings.

In [ ]:
m["Sum siblings"] = tt.agg.sum(
    m["Quantity.SUM"], scope=tt.scope.siblings(hier["Geography"])
)
cube.query(m["Sum siblings"], levels=[lvl["country"], lvl["city"]])

In [ ]:
m["Average siblings"] = tt.agg.mean(
    m["Quantity.SUM"], scope=tt.scope.siblings(hier["Geography"])
)
cube.query(m["Average siblings"], levels=[lvl["country"], lvl["city"]])

### Shift

`shift` allows to get the value of previous or next members on a level

In [ ]:
m["next quantity"] = tt.shift(m["Quantity.SUM"], on=lvl["Date"], offset=1)
cube.query(m["next quantity"], levels=[lvl["Date"]])

In [ ]:
cube.query(m["Quantity.SUM"], levels=[lvl["Date"]])

### Filter

`filter` allows you to only express the measure where certain conditions are met.  
You can only use conditions on **levels** with `filter`. If you want to use conditions involving measures you should use `where`.

You can combine conditions with the `&` bitwise operator.

In [ ]:
m["Filtered Quantity.SUM"] = tt.filter(
    m["Quantity.SUM"], (lvl["City"] == "Paris") & (lvl["Color"] == "red")
)
cube.query(
    m["Quantity.SUM"], m["Filtered Quantity.SUM"], levels=[lvl["Country"], lvl["Color"]]
)

### Where

`where` defines an if-then-else measure whose value depends on certain conditions. It is similar to numpy's [where](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html) function.

In [ ]:
# Label large amounts
m["Large quantities"] = tt.where(m["Quantity.SUM"] >= 3000, "large", "small")

# Double up UK's quantities
m["Double UK"] = tt.where(
    lvl["Country"] == "UK", 2 * m["Quantity.SUM"], m["Quantity.SUM"]
)

cube.query(
    m["Quantity.SUM"],
    m["Large quantities"],
    m["Double UK"],
    levels=[lvl["Country"], lvl["Color"]],
)

### At

`at` takes the value of the given measure shifted on one of the level to the given value.

The value can be a literal value or the value of another level

In [ ]:
m["Blue Quantity"] = tt.at(m["Quantity.SUM"], {lvl["Color"]: "blue"})
cube.query(m["Quantity.SUM"], m["Blue Quantity"], levels=[lvl["Color"]])

The measure is null if the level is not expressed

In [ ]:
cube.query(m["Quantity.SUM"], m["Blue Quantity"])

In a multilevel hierarchy, only the shifted level is changed.
For instance if we shift the country to "France", [Europe, UK] is shifted to [Europe,France] and [Asia,China] is shifted to [Asia,France] which does not exist.

In [ ]:
m["France Quantity Fail"] = tt.at(m["Quantity.SUM"], {lvl["country"]: "France"})
cube.query(
    m["Quantity.SUM"],
    m["France Quantity Fail"],
    levels=[lvl["continent"], lvl["country"]],
)

The expected value can be obtained by shifting both levels to the requested values as follow.

In [ ]:
m["France Quantity"] = tt.at(
    m["Quantity.SUM"], {lvl["country"]: "France", lvl["continent"]: "Europe"}
)
cube.query(
    m["Quantity.SUM"], m["France Quantity"], levels=[lvl["continent"], lvl["country"]]
)

## Other Functions

### Mathematical functions

`round`, `floor`, and `ceil`: closest, lower, and upper rounding of double values

In [ ]:
m["round"] = tt.round(m["Quantity.SUM"] / 1000) * 1000
m["floor"] = tt.floor(m["Quantity.SUM"] / 1000) * 1000
m["ceil"] = tt.ceil(m["Quantity.SUM"] / 1000) * 1000
cube.query(
    m["Quantity.SUM"], m["floor"], m["ceil"], m["round"], levels=[lvl["Country"]]
)

`exp`: Exponential is Euler's number `e` raised to the power of a double value.

In [ ]:
m["Zero"] = 0.0
m["One"] = tt.exp(m["Zero"])

`log`: the natural (base `e`) logarithm

In [ ]:
m["Log Quantity"] = tt.log(m["Quantity.SUM"])
cube.query(m["Log Quantity"], levels=[lvl["Country"]])

`log10`: the base 10 logarithm

In [ ]:
m["Log10 Quantity"] = tt.log10(m["Quantity.SUM"])
cube.query(m["Log10 Quantity"], levels=[lvl["Country"]])

`abs`: the absolute value of a given measure

In [ ]:
m["Negative"] = -1 * m["Quantity.SUM"]
m["abs"] = tt.abs(m["Negative"])
cube.query(m["abs"])

`sin`: the sinus value of a given measure

In [ ]:
m["Sinus Quantity"] = tt.sin(m["Quantity.SUM"])
cube.query(m["Sinus Quantity"], levels=[lvl["Country"]])

`cos`: the cosinus value of a given measure

In [ ]:
m["Cosinus Quantity"] = tt.cos(m["Quantity.SUM"])
cube.query(m["Cosinus Quantity"], levels=[lvl["Country"]])

`tan`: the tangent value of a given measure

In [ ]:
m["Tangent Quantity"] = tt.tan(m["Quantity.SUM"])
cube.query(m["Tangent Quantity"], levels=[lvl["Country"]])

`pow`: the calculated power value of a given measure

In [ ]:
m["Power int Quantity"] = m["Quantity.SUM"] ** 2
cube.query(m["Power int Quantity"], levels=[lvl["Country"]])

In [ ]:
m["Power float Quantity"] = m["Quantity.SUM"] ** 0.5
cube.query(m["Power float Quantity"], levels=[lvl["Country"]])

In [ ]:
m["Power measure Quantity"] = m["Quantity.SUM"] ** m["Quantity.SUM"]
cube.query(m["Power measure Quantity"], levels=[lvl["Country"]])

In [ ]:
m["Power neg Quantity"] = m["Quantity.SUM"] ** -3
cube.query(m["Power neg Quantity"], levels=[lvl["Country"]])

`sqrt`: the square root value of a given measure

In [ ]:
m["Sqrt Quantity"] = tt.sqrt(m["Quantity.SUM"])
cube.query(m["Sqrt Quantity"], levels=[lvl["Country"]])

### Time functions

`date_diff` is the number of days between 2 dates:

In [ ]:
from datetime import datetime

lvl = cube.levels
m["Days from today"] = tt.date_diff(datetime.now().date(), lvl["Date"])
cube.query(m["Days from today"], levels=[lvl["Date"]])

### Measure Folder

Measures can be put into folders to group them in the UI.

In [ ]:
for name in [
    "Quantity.SUM",
    "Quantity.MEAN",
    "Quantity.LONG",
    "Quantity.MAX",
    "Quantity.MIN",
    "Quantity.SHORT",
]:
    m[name].folder = "Quantity"

In [ ]:
m["Quantity.SUM"].folder

### Measure formatter

Measures have formatters describing how to represent them as strings. These formatters can be modified in Python or directly in the UI. Here are a few examples of what is possible, more documentation on how to write an MDX formatter can be found on [docs.microsoft.com](https://docs.microsoft.com/en-us/analysis-services/multidimensional-models/mdx/mdx-cell-properties-format-string-contents?view=sql-server-2017).

In [ ]:
# Convert to thousands
m["Quantity.SUM"].formatter = "DOUBLE[#,###,K]"

# Add a unit
m["Price.SUM"].formatter = "DOUBLE[#,###.00€]"

# Add more decimal figures
m["Quantity.MEAN"].formatter = "DOUBLE[#,###.0000]"

# Display as percent
m["Percent of parent"] = m["Quantity.SUM"] / m["Parent quantity"]
m["Percent of parent"].formatter = "DOUBLE[#,###.00%]"

![Launcher](images/ui-formatters.png)